In [ ]:
from pyimpspec import DataSet
import numpy as np 
data = np.loadtxt("KK_Test_Lucas.txt", skiprows=1)

frequencies = data[:, 0]
select = (frequencies > 10) & (frequencies < 10000)
frequencies = frequencies[select]
complex_impedance= data[select, 1] - 1j * data[select, 2]

dataset = DataSet(
    frequencies=frequencies,
    impedances=complex_impedance
)

import matplotlib.pyplot as plt 
%matplotlib widget 


# ax.set_xlim([0, 1e7])

In [ ]:
from pyimpspec import perform_kramers_kronig_test

result = perform_kramers_kronig_test(dataset)

In [ ]:
zr, negzimag = result.get_nyquist_data()

In [ ]:
f, real_res, imag_res = result.get_residuals_data()

color1 = "#0B9E7E"
fig, ax =plt.subplots(figsize=(5,2.5))
ax.plot(f, real_res, color=color1)
ax.set_ylim([-0.1, 0.1])
ax.set_ylabel(r'$\Delta_{\mathrm{Re}(Z)}$ (% of $|Z|$)')
ax.tick_params(axis='y', colors=color1)
ax.yaxis.label.set_color(color1)

color2 = "#C61193"
twinx = ax.twinx()
twinx.set_ylim([-0.1, 0.1])
twinx.plot(f, imag_res, color=color2)
twinx.set_ylabel(r'$\Delta_{\mathrm{Im}(Z)}$ (% of $|Z|$)')
twinx.tick_params(axis='y', colors=color2)
twinx.yaxis.label.set_color(color2)
ax.set_xscale('log')
ax.set_xlabel(r'$f$ (Hz)')
fig.tight_layout()
fig.savefig('kktest.pdf')

In [ ]:
fig, ax = plt.subplots()
ax.plot(complex_impedance.real, -complex_impedance.imag, color=color1, linewidth=1.5, label='COMSOL output')
ax.plot(zr, negzimag, color=color2, linestyle='--', linewidth=3, label='circuit fit')
ax.legend(frameon=False, loc='center left', bbox_to_anchor=(1, 0.5))
ax.set_aspect('equal')
ax.set_xlim([0, 1e5])
ax.set_xlabel(r"$\mathrm{Re}\;Z$ / $\Omega$")
ax.set_ylabel(r"$-\mathrm{Im}\;Z$ / $\Omega$")
fig.savefig('nyquist.pdf')

In [ ]:
import pandas as pd
df: pd.DataFrame = result.to_statistics_dataframe()
df

In [ ]:
from pyimpspec import calculate_drt

drt = calculate_drt(dataset, method="tr-nnls", lambda_value=1e-4)

drt.get_time_constants()

In [ ]:
from pyimpspec import perform_zhit

perform_zhit(dataset, admittance=True, num_procs=1)